In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
from torch_geometric.nn import GAE, GCNConv
from torch_geometric.utils import to_dense_adj
from tqdm import trange
from utils import evaluate_clustering, get_clusters, plot_training_history, plot_3d_clustering_comparison

# Adaptive Graph Encoder
From the paper [Adaptive Graph Encoder for Attributed Graph Embedding](https://arxiv.org/pdf/2007.01594)

## Loading the data
We use the Cora dataset, a standard benchmark dataset for clustering.

In [11]:
dataset = Planetoid(root='../data/Planetoid', name='Cora')
data = dataset[0]
data.train_mask = data.test_mask = data.val_mask = None
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708])

## Adaptive Graph Encoder (AdaGAE) model


In [12]:
A = to_dense_adj(data.edge_index)[0]
A_ = A + torch.eye(A.size(0))
D_ = torch.diag(torch.sum(A_, dim=1))
L_ = D_ - A_
D_inv_sqrt = torch.diag(1 / torch.sqrt(torch.diag(D_)))
L_sym = D_inv_sqrt @ L_ @ D_inv_sqrt
L_sym.shape

torch.Size([2708, 2708])

In [13]:
k = 2/3  # from the paper (best value for cora)
H = torch.eye(L_sym.size(0)) - k * L_sym
H, H.shape

(tensor([[0.5000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5000, 0.1361,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.1361, 0.4444,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.6667, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4667, 0.1333],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1333, 0.4667]]),
 torch.Size([2708, 2708]))

In [14]:
X_ = H.pow(8) @ data.x  # t=8 from the paper
X_, X_.shape

(tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 2.4387e-07,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 torch.Size([2708, 1433]))

In [15]:
S = X_ @ X_.T / X_.norm(p=2)
S, S.shape

(tensor([[3.6707e-05, 4.0928e-06, 1.5924e-06,  ..., 4.0740e-05, 4.6988e-06,
          4.6985e-06],
         [4.0928e-06, 9.3958e-05, 4.7989e-06,  ..., 8.1669e-05, 2.3599e-06,
          2.3560e-06],
         [1.5924e-06, 4.7989e-06, 1.1775e-05,  ..., 3.1836e-05, 9.1771e-07,
          9.1675e-07],
         ...,
         [4.0740e-05, 8.1669e-05, 3.1836e-05,  ..., 7.3334e-03, 3.6351e-08,
          2.3459e-05],
         [4.6988e-06, 2.3599e-06, 9.1771e-07,  ..., 3.6351e-08, 1.8931e-05,
          2.7065e-06],
         [4.6985e-06, 2.3560e-06, 9.1675e-07,  ..., 2.3459e-05, 2.7065e-06,
          1.7579e-05]]),
 torch.Size([2708, 2708]))

In [16]:
X_.size(1)

1433

In [17]:
X_.norm(p=2)

tensor(3.7417)

In [17]:
model = nn.Sequential(
	nn.Linear(X_.size(1), 16),
	nn.ReLU(),
	nn.Linear(16, 8),
)